In [1]:
from sec_edgar_downloader import Downloader
import datetime
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#nltk.download('all')
from bs4 import BeautifulSoup
import re
import pysentiment2
import math

In [2]:
def get_item_7a_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        raw_10k = file.read()
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
    document = {}

    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-K':
            document[doc_type] = raw_10k[doc_start:doc_end]

    regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|7A|7|8)\.{0,1})|(ITEM\s(1A|1B|7A|7|8))')

    matches = regex.finditer(document['10-K'])
    matches = regex.finditer(document['10-K'])
    test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

    test_df.columns = ['item', 'start', 'end']
    test_df['item'] = test_df.item.str.lower()
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace(' ','',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace('>','',regex=True,inplace=True)

    pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

    pos_dat.set_index('item', inplace=True)

    item_7_raw = document['10-K'][pos_dat['start'].loc['item7a']:pos_dat['start'].loc['item8']]
    item_7_content = BeautifulSoup(item_7_raw, 'lxml')
    #print(item_1a_content.prettify()[0:1000])
    return item_7_content.get_text("\n\n")

In [3]:
def sentiment(text):
    s = SentimentIntensityAnalyzer()
    return s.polarity_scores(text)

In [4]:
sentiment(get_item_7a_text("./V-16-10-K-1.txt"))

{'neg': 0.051, 'neu': 0.803, 'pos': 0.146, 'compound': 1.0}

In [5]:
VISAtext = get_item_7a_text("./V-16-10-K-1.txt")